In [75]:
import pymysql

In [76]:
import getpass

# Connect to server
host="localhost"
user="root"
password=getpass.getpass("Please enter SQL local server password :D")

connection = pymysql.connect(
    host=host,
    user=user,
    password=password,
)
print("Connection successful!")

cursor = connection.cursor()

Connection successful!


In [77]:
# SQL reading functions

def perform_queries_from_file(query_file):
    # Open and read the SQL file
    with open(query_file, 'r', encoding='utf-8') as file:
        sql_queries = file.read()

    # Default delimiter
    delimiter = ';'

    # Split the SQL file into lines for processing
    lines = sql_queries.splitlines()
    current_query = ""

    # Iterate over each line in the SQL file
    for line in lines:
        stripped_line = line.strip()

        # Check for delimiter change
        if stripped_line.startswith("DELIMITER"):
            delimiter = stripped_line.split()[1]
            continue

        # Add the current line to the query
        current_query += line + "\n"

        # Check if the query ends with the current delimiter
        if current_query.strip().endswith(delimiter):
            try:
                # Remove the delimiter and execute the query
                query_to_execute = current_query.strip()[:-len(delimiter)].strip()
                if query_to_execute:  # Only execute non-empty queries
                    # cursor = db_connection.cursor()
                    cursor.execute(query_to_execute)
                    connection.commit()
                    print("Query executed successfully!")
            except Exception as e:
                print("Error executing query:", str(e))
                print(current_query)

            # Reset the current query after execution
            current_query = ""

    # If there's any remaining query after processing all lines, execute it
    if current_query.strip():
        try:
            # cursor = db_connection.cursor()
            cursor.execute(current_query.strip())
            connection.commit()
            print("Query executed successfully!")
        except Exception as e:
            print("Error executing query:", str(e))
            print(current_query)

def show_query_output(output):
  for tuple in output:
    print(tuple)

In [78]:
# Test to see if connected properly
cursor = connection.cursor()

cursor.execute("SHOW DATABASES")
output = cursor.fetchall()
print(output)


(('airline',), ('hw1',), ('information_schema',), ('mysql',), ('performance_schema',), ('sakila',), ('sys',), ('uni',), ('university',))


# Database initialization

In [79]:
cursor.execute("DROP DATABASE IF EXISTS airline;")
cursor.execute("CREATE DATABASE airline")
cursor.execute("USE airline")

0

In [80]:
print("Creating tables")
perform_queries_from_file("Updated DDL.sql")
print("Filling tables")
perform_queries_from_file("master_fill.sql")

# print("Filling countries")
# perform_queries_from_file("fill_countries.sql")
# print("Filling airlines")
# perform_queries_from_file("fill_airlines.sql")
# print("Filling airports")
# perform_queries_from_file("fill_airports.sql")
# print("Filling crew")
# perform_queries_from_file("fill_crew.sql")
# print("Filling rest")
# perform_queries_from_file("MadeUpData.sql")

Creating tables
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Filling tables
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query execut

# Json reader

In [81]:
import json
from datetime import datetime, timedelta
import re

# Function to convert travel time to duration in minutes
def convert_travel_time_to_minutes(travel_time):

    # Remove "Overnight" from the travel time if present
    travel_time = travel_time.replace("Overnight", "").strip()

    # Handle case: "X hr Y min"
    match = re.match(r"(\d+)\s*hr\s*(\d+)\s*min", travel_time)
    if match:
        hours = int(match.group(1))
        minutes = int(match.group(2))
        return hours * 60 + minutes
    
    # Handle case: "X hr" (only hours)
    match = re.match(r"(\d+)\s*hr", travel_time)
    if match:
        hours = int(match.group(1))
        return hours * 60
    
    # Handle case: "X min" (only minutes)
    match = re.match(r"(\d+)\s*min", travel_time)
    if match:
        minutes = int(match.group(1))
        return minutes
    raise ValueError(f"Invalid travel time format: {travel_time}")

# Function to convert time string (e.g., "6:39 PM") to 24-hour format datetime
def convert_time_to_datetime(date_str, time_str):
    normalized_time_str = time_str.replace('â€¯', ' ').replace('Ã¢â‚¬Â¯', ' ').strip()
    
    # Check for additional days specified with "+X"
    additional_days = 0
    match = re.search(r"\+(\d+)", normalized_time_str)
    if match:
        additional_days = int(match.group(1))
        # Remove the "+X" part from the time string
        normalized_time_str = re.sub(r"\+\d+", "", normalized_time_str).strip()
    
    # Convert the time string to a datetime object
    time_obj = datetime.strptime(normalized_time_str, "%I:%M %p")
    base_date = datetime.strptime(date_str, "%Y-%m-%d")
    return base_date + timedelta(days=additional_days, hours=time_obj.hour, minutes=time_obj.minute)

# Read json 
def jsonToSql(jsonFileName):
    with open(jsonFileName, "r") as file: # CHANGE JSON NAME HERE -----------------------------
        flight_data = json.load(file)

    for flight in flight_data:
        try:
            # Extract necessary information from the JSON
            flight_num = flight["flight_num"]
            departure_airport_code = flight["departure_airport_code"]
            arrival_airport_code = flight["arrival_airport_code"]
            departure_time = flight["departure_time"]
            arrival_time = flight["arrival_time"]
            travel_time = flight["travel_time"]
            airline_code = flight["airline_code"]
            departure_date = flight["query_date"]

            # Convert travel_time to duration in minutes
            duration = convert_travel_time_to_minutes(travel_time)

            # Convert departure_time and arrival_time to datetime objects
            departure_datetime = convert_time_to_datetime(departure_date, departure_time)
            arrival_datetime = convert_time_to_datetime(departure_date, arrival_time)

            # Prepare the SQL insert statement
            insert_sql = """
                INSERT INTO flight (flight_id, departure_airport, arrival_airport, departure_datetime, arrival_datetime, duration, airline_id, international, tail_no)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            international = None
            tail_no = None  # Set to None as per your requirement

            # print(flight_num, departure_airport_code, arrival_airport_code, 
            #     departure_datetime, arrival_datetime, duration, 
            #     airline_code, international, tail_no)


            # Execute the insert statement
            cursor.execute(insert_sql, (
                flight_num, departure_airport_code, arrival_airport_code, 
                departure_datetime, arrival_datetime, duration, 
                airline_code, international, tail_no
            ))

            # Commit the transaction
            connection.commit()
            print(f"Flight {flight_num} added successfully!")

        except Exception as e:
            # Print an error message but continue with the next flight
            print(f"Failed to add flight {flight['flight_num']}: {e}")

jsonToSql('test.json')

Failed to add flight SY 286: (1062, "Duplicate entry 'SY 286-2024-12-04 18:39:00' for key 'flight.PRIMARY'")
Failed to add flight AA 345: (1062, "Duplicate entry 'AA 345-2024-12-04 18:39:00' for key 'flight.PRIMARY'")
Failed to add flight AA 456: (1062, "Duplicate entry 'AA 456-2024-12-04 18:39:00' for key 'flight.PRIMARY'")
Failed to add flight AA 567: (1062, "Duplicate entry 'AA 567-2024-12-04 18:39:00' for key 'flight.PRIMARY'")
Failed to add flight AA 678: (1062, "Duplicate entry 'AA 678-2024-12-04 18:39:00' for key 'flight.PRIMARY'")
Failed to add flight AA 789: (1062, "Duplicate entry 'AA 789-2024-12-04 18:39:00' for key 'flight.PRIMARY'")
Failed to add flight AA 890: (1062, "Duplicate entry 'AA 890-2024-12-04 18:39:00' for key 'flight.PRIMARY'")
Flight UA 1701 added successfully!
Flight UA 2288 added successfully!
Flight NK 2074 added successfully!
Flight WN 966 added successfully!
Flight WN 1937 added successfully!
Failed to add flight 9C 8891: (1452, 'Cannot add or update a ch

In [82]:
# cursor.execute("delete from flight;")
# connection.commit()

In [83]:
# # Just in case you need to close connection
# if connection.open:
#         connection.close()
#         print("Connection closed.")

# Google Flights Scraper

In [84]:
# Scraper
from playwright.async_api import async_playwright
from tenacity import retry, stop_after_attempt, wait_fixed
from dataclasses import dataclass
from typing import List, Optional
from datetime import datetime
import json
import os
import nest_asyncio

SELECTORS = {
    "departure_time": 'div.b0EVyb.YMlIz.y52p7d[aria-label^="Departure time"]',
    "flight_num": 'span.Xsgmwe.sI2Nye',
    "airline": 'span.Xsgmwe',
    "price": 'span[data-gs][aria-label$="US dollars"]',
    'departure_time': 'div.b0EVyb.YMlIz.y52p7d',
    'travel_time': 'div.P102Lb.sSHqwe.y52p7d',
    'arrival_time': 'div.OJg28c.YMlIz.y52p7d',
    'arrival_airport': 'div.FY5t7d.tdMWuf.y52p7d',
    'departure_airport': 'div.ZHa2lc.tdMWuf.y52p7d',
    'departure_airport_code': 'div.ZHa2lc.tdMWuf.y52p7d span[dir="ltr"]',
    'arrival_airport_code': 'div.FY5t7d.tdMWuf.y52p7d span[dir="ltr"]',
}
# successfully_loaded = [['PUW', 'SEA'], ['SEA', 'PUW'], ['SEA', 'LAX'], ['LAX', 'SEA'], ['SEA', 'MSP'], ['MSP', 'SEA'], ['MSP', 'DLH'], ['DLH', 'MSP'], ['SEA', 'IST'], ['IST', 'SEA'], ['SHA', 'PEK'], ['PEK', 'SHA'], ['SEA', 'PEK'], ['PEK', 'SEA'], ['LAX', 'PEK']]
# airport_routes = [['PUW', 'SEA']]
# query_dates = ['2024-12-10'] # flee the country the day before the final

async def scrape_google_flights(airport_routes, query_dates):

    async with async_playwright() as p:
        # Launch browser
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()

        async def init_setup(page):
            ticket_type_div = page.locator("div.VfPpkd-TkwUic[jsname='oYxtQd']").first
            await ticket_type_div.click()
            await page.wait_for_timeout(300)
            await page.wait_for_selector("ul[aria-label='Select your ticket type.']")
            await page.locator("li").filter(has_text="One way").nth(0).click()
            await page.wait_for_timeout(200)
            await page.locator("body").click()
            await page.wait_for_timeout(250)

        async def departure_arrival(page, depart_airport, arrive_airport):
            await page.keyboard.press("Tab")
            await page.keyboard.press("Tab")
            await page.keyboard.press("Tab")
            await page.keyboard.press("Tab")
            await page.wait_for_timeout(200)
            await page.keyboard.type(depart_airport)
            await page.wait_for_timeout(200)
            await page.keyboard.press("Tab")
            await page.keyboard.press("Tab")
            await page.wait_for_timeout(350)

            # Fill destination
            await page.keyboard.type(arrive_airport)
            await page.wait_for_timeout(300)
            await page.keyboard.press("Tab")
            await page.keyboard.press("Tab")
            await page.wait_for_timeout(700)

        async def select_day(page, date):
            await page.keyboard.type(date)
            await page.wait_for_timeout(700)
            await page.keyboard.press("Enter")
            await page.keyboard.press("Tab")
            await page.keyboard.press("Tab")
            await page.keyboard.press("Enter")
            await page.wait_for_timeout(4000)
        
        async def select_nonstop_flights(page):
            # Click the "Stops" button
            stops_button = page.locator('button[aria-label="Stops, Not selected"]')
            await stops_button.click()
            await page.wait_for_timeout(300)
            # Select "Nonstop only" option
            nonstop_option = page.locator('input[aria-label="Nonstop only"]').first
            await nonstop_option.click()
            await page.wait_for_timeout(300)
            await page.keyboard.press("Escape")
        
        async def main_query_function(page, depart_airport, arrive_airport, date):
            try:
                await init_setup(page)
                await departure_arrival(page, depart_airport, arrive_airport)
                await select_day(page, date)
                print("Page loaded successfully")
                await select_nonstop_flights(page)
            except Exception as e:
                print(f"Error: {e}")
                    
        async def load_all_available_flights(page):
            while True:
                try:
                    # Wait for the "more flights" button
                    more_button = await page.wait_for_selector(
                    'button[aria-label*="more flights"]', timeout=3000
                    )
                    if more_button:
                        await more_button.click()
                        # Wait for new flights to load
                        await page.wait_for_timeout(500)
                    else:
                        break
                except:
                    # No more "Show more" button found
                    break

        async def click_expand_buttons(page):
            expand_buttons = await page.query_selector_all('button[aria-label*="Flight details"]')
            for button in expand_buttons:
                try:
                    await button.click()
                    print(f'clicked button: {button}')
                    await page.wait_for_timeout(3)
                except:
                    # If clicking fails, break
                    print('did not click button, break')
                    break

        async def extract_text(element):
            if element:
                return await element.text_content()
            else:
                return "N/A"

        async def extract_flight_data(page, date):
            print('beginning flight data extraction')
            try:
                await page.wait_for_selector("div.c257Jb.QwxBBf.eWArhb", timeout=3000)

                # Now extract all flight data
                flights = await page.query_selector_all("div.c257Jb.QwxBBf.eWArhb")

                flights_data = []
                for flight in flights:
                    print(f'flight: {flight}')
                    flight_info = {}
                    for key, selector in SELECTORS.items():
                        element = await flight.query_selector(selector)
                        flight_info[key] = await extract_text(element)
                    flight_info['query_date'] = date

                    print(flight_info)

                    flights_data.append(flight_info)
                print(flights_data)

                # extend json if already exists, otherwise json dump
                if os.path.exists("flight_data.txt"):
                    with open("flight_data.txt", "r") as f:
                        existing_data = json.load(f)
                    existing_data.extend(flights_data)
                    with open("flight_data.txt", "w") as f:
                        json.dump(existing_data, f, indent=4)
                else:
                    with open("flight_data.txt", "w") as f:
                        json.dump(flights_data, f, indent=4)

            except Exception as e:
                raise Exception(f"Failed to extract flight data: {str(e)}")

        # print("Navigating to Google Flights...")
        # await page.goto("https://www.google.com/flights", timeout=10000)
        # print("Calling main query function")
        # await main_query_function(page, airport_routes[0][0], airport_routes[0][1], query_dates[0])
        # await load_all_available_flights(page)
        # await click_expand_buttons(page)
        # await extract_flight_data(page, query_dates[0])

        for date in query_dates:
            for route in airport_routes:
                await page.goto("https://www.google.com/flights", timeout=10000)
                depart_airport, arrive_airport = route[0], route[1]
                await main_query_function(page, depart_airport, arrive_airport, date)
                print('loading all available flights')
                await load_all_available_flights(page)
                print('clicking expand buttons')
                await click_expand_buttons(page)
                print('extracting flight data')
                await extract_flight_data(page, date)


In [85]:
# Function to clean scrape output file
def cleanTxtFile(file_path):
    # Read the JSON data from the file
    with open(file_path, 'r', encoding='utf-8') as file:
        flight_data = json.load(file)

    # Clean the flight data
    for flight in flight_data:
        flight['flight_num'] = flight['flight_num'].replace('\u00a0', ' ')
        flight['departure_airport_code'] = flight['departure_airport_code'].replace('(', '').replace(')', '')
        flight['arrival_airport_code'] = flight['arrival_airport_code'].replace('(', '').replace(')', '')
        flight['airline_code'] = flight['flight_num'].split(' ')[0]
        flight['travel_time'] = flight['travel_time'].replace('Travel time: ', '')
        
    # Write the cleaned data back to the file
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(flight_data, file, ensure_ascii=False, indent=4)

async def scrapeFunction():
    # Get scrape parameters
    array_airport_routes = []
    while True:
        route = input("Enter route (e.g., JFK,LAX) or 'done': ").strip()
        if route.lower() == 'done':
            break
        try:
            departure_airport, arrival_airport = route.split(',')
            array_airport_routes.append([departure_airport.strip(), arrival_airport.strip()])
        except ValueError:
            print("Invalid format. Please enter routes as 'departure_airport,arrival_airport'.")
    
    array_query_dates = []
    while True:
        date = input("Enter departure date (e.g., 2024-12-10) or 'done': ").strip()
        if date.lower() == 'done':
            break
        try:
            datetime.strptime(date, "%Y-%m-%d")
            array_query_dates.append(date)
        except ValueError:
            print("Invalid date format. Please enter dates as 'YYYY-MM-DD'.")

    print("\nCollected Routes:")
    print(array_airport_routes)
    print("\nCollected Query Dates:")
    print(array_query_dates)

    # Run scraper
    nest_asyncio.apply()
    await scrape_google_flights(array_airport_routes, array_query_dates)

    # Format file
    cleanTxtFile('flight_data.txt')

    # Add to flight table
    jsonToSql('flight_data.txt')

# await scrapeFunction()

In [86]:
# jsonToSql('flight_data.txt')

# Queries :)

In [87]:
perform_queries_from_file("Views.sql")
perform_queries_from_file("Procedures.sql")
connection.commit()

Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed successfully!
Query executed

### Tables, views

In [88]:
from tabulate import tabulate

# Function to display tables
def makeTable(output1):
    columns = [desc[0] for desc in cursor.description]
    if output1:
        print(tabulate(output1, headers=columns, tablefmt="pretty"))
    else:
        print("No data available.")

# Basic table viewing
def viewFlights():
    print("\nFetching all flights...")
    cursor.execute('Select * FROM flight;')
    output = cursor.fetchall()
    makeTable(output)

def viewFlightsByAirline():
    airline_id = input("Enter airline_id (e.g. AA): ").strip()
    print(f"\nFetching all flights from {airline_id}...")
    cursor.execute('SELECT * FROM flight WHERE airline_id = %s;', (airline_id))
    output = cursor.fetchall()
    makeTable(output)

def viewFlightsToFrom():
    departure_airport = input("Enter departure_airport (e.g. AAA, or leave blank): ").strip()
    arrival_airport = input("Enter arrival_airport (e.g. AAA, or leave blank): ").strip()
    if not departure_airport and not arrival_airport:
        print('You have not entered any airports.')
        return
    elif not arrival_airport:
        print(f'Fetching all flights out of {departure_airport}')
        cursor.execute('SELECT flight_id, arrival_airport, departure_datetime, arrival_datetime, duration, airline_id, international, tail_no FROM flight WHERE departure_airport = %s', (departure_airport))
    elif not departure_airport:
        print(f'Fetching all flights to {arrival_airport}')
        cursor.execute('SELECT flight_id, departure_airport, departure_datetime, arrival_datetime, duration, airline_id, international, tail_no FROM flight WHERE arrival_airport = %s', (arrival_airport))
    else:
        print(f'Fetching all flights from {departure_airport} to {arrival_airport}')
        cursor.execute('SELECT flight_id, departure_datetime, arrival_datetime, duration, airline_id, international, tail_no FROM flight WHERE departure_airport = %s AND arrival_airport = %s', (departure_airport, arrival_airport))
    output = cursor.fetchall()
    makeTable(output)

def viewCrewByAirline():
    airline_id = input("Enter airline_id (e.g. AA): ").strip()
    print(f"\nFetching crew working for {airline_id}...")
    cursor.execute('SELECT crew_id, CONCAT(f_name, " ", l_name) as name, role, license_no, airline_id, salary, code FROM crew WHERE airline_id = %s;', (airline_id,))
    output = cursor.fetchall()
    makeTable(output)

def viewAircraftByAirline():
    airline_id = input("Enter airline_id (e.g. AA): ").strip()
    print(f"\nFetching aircraft under {airline_id}...")
    cursor.execute('SELECT * FROM aircraft WHERE airline_id = %s;', (airline_id,))
    output = cursor.fetchall()
    makeTable(output)
#viewAircraftByAirline()

def viewMaintenanceRecords():
    print("\nFetching all maintenance records...")
    cursor.execute('Select * FROM maintenance_record;')
    output = cursor.fetchall()
    makeTable(output)
#viewMaintenanceRecords()

def viewMaintenanceRecordsByAircraft():
    tail_no = input("Enter airline_id (e.g. AA): ").strip()
    print(f"\nFetching all maintenance records for aircraft {tail_no}...")
    cursor.execute('Select * FROM maintenance_record WHERE tail_no = %s;', (tail_no))
    output = cursor.fetchall()
    makeTable(output)
#viewMaintenanceRecordsByAircraft()


def UpcomingFlightsForPassenger():
    passenger_id = input('Enter name (first last): ').strip()
    print(f"\nFetching all upcoming flights for passenger {passenger_id}...")
    cursor.execute('SELECT * FROM UpcomingFlightsForPassenger WHERE passenger_id = %s;', (passenger_id))
    output = cursor.fetchall()
    makeTable(output)

def showAssigned():
    print("\nFetching crew assignment information...")
    cursor.execute("SELECT * FROM assigned;")
    output = cursor.fetchall()
    makeTable(output)

def showAssignedByFlight():
    flight_id = input("Enter flight ID: ").strip()
    departure_datetime_str = input("Enter departure datetime (YYYY-MM-DD HH:MM): ").strip()
    departure_datetime = datetime.strptime(departure_datetime_str, "%Y-%m-%d %H:%M")
    print(f"\nFetching crew assignment information for {flight_id} leaving on {departure_datetime}...")
    cursor.execute("SELECT * FROM assigned WHERE flight_id = %s AND departure_datetime = %s;", (flight_id, departure_datetime))
    output = cursor.fetchall()
    makeTable(output)

def countAirportsByCountry():
    cursor.execute('''SELECT country, COUNT(country) AS airport_count
                    FROM airport as a
                    GROUP BY country
                    ORDER BY airport_count DESC;
                   ''')
    output = cursor.fetchall()
    makeTable(output)

def mostConnectedCountries():
    count = int(input("Enter count (top _ connected countries): ").strip())
    print(f"\nTop {count} connected countries by number of airlines: ")
    cursor.execute('''SELECT country, COUNT(country) as airline_count
                    FROM airline as airl
                    GROUP BY country
                    ORDER BY airline_count DESC
                    LIMIT %s;''', (count))
    output = cursor.fetchall()
    makeTable(output)

def crewSalaryMetrics():
    print("Fetching crew salary metrics...")
    cursor.execute('SELECT * FROM Crew_Salary_Metrics;')
    output = cursor.fetchall()
    makeTable(output)

def crewSalaryMetricsByRole():
    print("\nFetching crew salary metrics by role...")
    cursor.execute('SELECT * FROM Crew_Salary_Metrics_By_Role;')
    output = cursor.fetchall()
    makeTable(output)

def shortestLayovers():
    count = int(input("Enter count (top _ shortest layovers): ").strip())
    print(f"\nTop {count} shortest layovers: ")
    cursor.execute('''WITH flight_pairs AS (
                        SELECT f1.flight_id AS first_flight_id, f1.arrival_airport AS layover_airport, f2.flight_id AS second_flight_id, TIMESTAMPDIFF(MINUTE, f1.arrival_datetime, f2.departure_datetime) AS layover_duration
                        FROM flight f1 JOIN flight f2
                        ON f1.arrival_airport = f2.departure_airport AND f1.arrival_datetime < f2.departure_datetime
                        WHERE TIMESTAMPDIFF(MINUTE, f1.arrival_datetime, f2.departure_datetime) > 0
                    )
                    SELECT first_flight_id, second_flight_id, layover_airport, layover_duration
                    FROM flight_pairs
                    ORDER BY layover_duration ASC
                    LIMIT %s;''', (count))
    output = cursor.fetchall()
    makeTable(output)


### Procedures, functions

In [89]:
import networkx as nx

# Creates entry for booking table
def AddPassengerBooking():
    print("\nAdding a new booking...")
    flight_id = input("Enter flight ID (e.g. AB 123): ").strip()
    departure_datetime_str = input("Enter departure datetime (YYYY-MM-DD HH:MM): ").strip()
    passenger_id = int(input("Enter passenger ID (e.g.): ").strip())
    booking_date = input("Enter booking date (YYYY-MM-DD): ").strip()
    travel_class = input("Enter travel class (Economy/Business): ").strip()
    seat_number = input("Enter seat number: ").strip()
    carry_on = bool(input("Carry-on luggage (true/false): ").strip().lower())
    checked = int(input("Number of checked bags: ").strip())
    status = input("Enter booking status (Confirmed/Canceled): ").strip()
    price = float(input("Enter price: ").strip())

    parameters = (flight_id, datetime.strptime(departure_datetime_str, "%Y-%m-%d %H:%M"), passenger_id, datetime.strptime(booking_date, "%Y-%m-%d"), travel_class, seat_number, carry_on, checked, status, price)
    try:
        cursor.callproc("AddPassengerBooking", parameters)
    except Exception as e:
        print("Error:", str(e))
    connection.commit()

# Shortest path code
def findShortestPath(departure_airport, arrival_airport):
    """
    Finds the shortest path between two airports using flight connections.

    :param departure_airport: IATA code of the departure airport
    :param arrival_airport: IATA code of the arrival airport
    :param db_connection: Active MySQL database connection
    :return: List of flights in the shortest path or an error message
    """
    try:

        # Query to retrieve all flights as edges in the graph
        cursor.execute("""
        SELECT departure_airport, arrival_airport, duration, flight_id 
        FROM flight
                       """)
        flights = cursor.fetchall()

        # Create a directed graph
        graph = nx.DiGraph()

        # Add edges to the graph
        for flight in flights:
            dep_airport, arr_airport, duration, flight_id = flight
            graph.add_edge(dep_airport, arr_airport, weight=duration, flight_id=flight_id)

        # Use Dijkstra's algorithm to find the shortest path
        try:
            path = nx.shortest_path(graph, source=departure_airport, target=arrival_airport, weight='weight')
            # Collect detailed flight information for the path
            route = []
            for i in range(len(path) - 1):
                flight_details = graph[path[i]][path[i + 1]]
                route.append({
                    "from": path[i],
                    "to": path[i + 1],
                    "flight_id": flight_details["flight_id"],
                    "duration": flight_details["weight"]
                })
            return route

        except nx.NetworkXNoPath:
            return f"No route found between {departure_airport} and {arrival_airport}."
        except nx.NodeNotFound:
            return f"One or both airports not found in the database."

    except pymysql.MySQLError as e:
        return f"Database error: {str(e)}"
# Function to implement
def returnShortestPath():
    # Testing pathfinding
    departure = input("Enter departure airport code:").strip().upper()
    arrival = input("Enter arrival airport code:").strip().upper()
    result = findShortestPath(departure, arrival)

    if isinstance(result, str):
        print(f"\nShortest Route from {departure} to {arrival}:")
        print(result)  # Error message
    else:
        print(f"Shortest Route from {departure} to {arrival}:")
        for leg in result:
            print(f"Flight from {leg['from']} to {leg['to']} (Flight ID: {leg['flight_id']}) - Duration: {leg['duration']} mins")


def assignAvailableCrew():
    flight_id = input("Enter flight ID: ").strip()
    departure_datetime_str = input("Enter departure datetime (YYYY-MM-DD HH:MM): ").strip()
    parameters = (flight_id, datetime.strptime(departure_datetime_str, "%Y-%m-%d %H:%M"))
    try:
        cursor.callproc("assignAvailableCrew", parameters)
        output = cursor.fetchall()
        makeTable(output)
    except Exception as e:
        print("Error:", str(e))
    connection.commit()

def assignPlane():
    flight_id = input("Enter flight ID: ").strip()
    departure_datetime_str = input("Enter departure datetime (YYYY-MM-DD HH:MM): ").strip()
    departure_datetime = datetime.strptime(departure_datetime_str, "%Y-%m-%d %H:%M")
    cursor.execute('CALL assignPlane(%s, %s, @o_tail_no, @message);', (flight_id, departure_datetime))
    cursor.execute('SELECT @o_tail_no, @message;')
    output = cursor.fetchall()
    makeTable(output)
    connection.commit()

def removeOldAircraft():
    print("\nRemoving old aircraft...")
    try:
        cursor.callproc("RemoveOldAircraft")
    except Exception as e:
        print("Error:", str(e))
    connection.commit()
    

def giveAllRolesRaises():
    percent_increase = float(input("Enter percentage increase: ").strip())
    try:
        cursor.callproc("giveAllRolesRaises", (percent_increase,))
    except Exception as e:
        print("Error:", str(e))
    connection.commit()

def updateSalaryByRole():
    role_name = input("Enter role name: ").strip()
    percent_increase = float(input("Enter percentage increase: ").strip())
    parameters = (role_name, percent_increase)
    try:
        cursor.callproc("UpdateSalaryByRole", parameters)
    except Exception as e:
        print("Error:", str(e))
    connection.commit()

def getHighLowSalaryWithCursor():
    cursor.execute("CALL GetHighLowSalaryWithCursor(@highest_salary, @lowest_salary);")
    cursor.execute('SELECT @highest_salary as HighestSalary, @lowest_salary as LowestSalary;')
    output = cursor.fetchall()
    makeTable(output)
    connection.commit()

def getBusiestAirport():
    print("Fetching busiest airport...")
    cursor.execute('SELECT GetBusiestAirportString() AS busiest_airport;')
    output = cursor.fetchall()
    makeTable(output)


In [90]:
# Test pathfinding
departure = 'LAX'
arrival = 'DLH'
result = findShortestPath(departure, arrival)
# lax to sea to msp to dlh

if isinstance(result, str):
    print(result)  # Error message
else:
    print("Shortest Route:")
    for leg in result:
        print(f"Flight from {leg['from']} to {leg['to']} (Flight ID: {leg['flight_id']}) - Duration: {leg['duration']} mins")

Shortest Route:
Flight from LAX to SEA (Flight ID: UA 2390) - Duration: 175 mins
Flight from SEA to MSP (Flight ID: SY 286) - Duration: 200 mins
Flight from MSP to DLH (Flight ID: DL 4150) - Duration: 66 mins


# Terminal

In [91]:
cursor = connection.cursor()

# Login
def authenticate_user(username, password):
    # Simulate a check for credentials (replace with actual authentication logic)
    if username == "admin" and password == "123":
        return True
    return False

# A terminal for the airline database access
def airline_terminal():
    while True:
        print("\nAirline Terminal")
        print("Choose an action (0-23):")

        choice = input("\nEnter your choice (0-23): ").strip()
        if choice == "1":
            viewFlights()
        elif choice == "2":
            viewFlightsByAirline()
        elif choice == "3":
            viewFlightsToFrom()
        elif choice == "4":
            returnShortestPath()
        elif choice == "5":
            viewCrewByAirline()
        elif choice == "6":
            viewMaintenanceRecords()
        elif choice == "7":
            viewMaintenanceRecordsByAircraft()
        elif choice == "8":
            UpcomingFlightsForPassenger()
        elif choice == "9":
            showAssigned()
        elif choice == "10":
            showAssignedByFlight()
        elif choice == "11":
            countAirportsByCountry()
        elif choice == "12":
            mostConnectedCountries()
        elif choice == "13":
            crewSalaryMetrics()
        elif choice == "14":
            crewSalaryMetricsByRole()
        elif choice == "15":
            getHighLowSalaryWithCursor()
        elif choice == "16":
            shortestLayovers()
        elif choice == "17":
            getBusiestAirport()
        elif choice == "18":
            AddPassengerBooking()
        elif choice == "19":
            assignAvailableCrew()
        elif choice == "20":
            assignPlane()
        elif choice == "21":
            removeOldAircraft()
        elif choice == "22":
            giveAllRolesRaises()
        elif choice == "23":
            updateSalaryByRole()
        elif choice == "0":
            print("Exiting airline terminal.")
            break
        else:
            print("Please try again.")

async def main():
    while True:
        print("0 - Exit")
        print("1 - Scrape flights")
        print("2 - Access airline database")
        choice = input("Enter your choice (0/1/2): ").strip()

        # Scrape flights -----------------------------
        if choice == "1":
            await scrapeFunction()
        # Access airline database ------------------------------------
        elif choice == "2":
            print("\nLogin to the airline database...")
            print("Enter username")
            username = input("Username: ").strip()
            print("Enter password")
            password = getpass.getpass("Password: ")  # Masked input for password

            if authenticate_user(username, password):
                print("Login successful!")
                # Simulate a database cursor
                airline_terminal()
            else:
                print("Invalid username or password. Please try again.")

        # Exit terminal ------------------------
        elif choice == "0":
            print("Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    await main()

0 - Exit
1 - Scrape flights
2 - Access airline database



Login to the airline database...
Enter username
Enter password
Login successful!

Airline Terminal
Choose an action (0-23):
Shortest Route from LAX to DLH:
Flight from LAX to SEA (Flight ID: UA 2390) - Duration: 175 mins
Flight from SEA to MSP (Flight ID: SY 286) - Duration: 200 mins
Flight from MSP to DLH (Flight ID: DL 4150) - Duration: 66 mins

Airline Terminal
Choose an action (0-23):

Shortest Route from PVG to BWI:
No route found between PVG and BWI.

Airline Terminal
Choose an action (0-23):

Shortest Route from PVG to LAX:
No route found between PVG and LAX.

Airline Terminal
Choose an action (0-23):

Fetching all flights...
+-----------+-------------------+-----------------+---------------------+---------------------+----------+------------+---------------+---------+
| flight_id | departure_airport | arrival_airport | departure_datetime  |  arrival_datetime   | duration | airline_id | international | tail_no |
+-----------+-------------------+-----------------+---------------